# Restaurant Entity-Relation Neo4j Knowledge Graph

This notebook demonstrates the creation and querying of a knowledge graph for restaurant data using Neo4j. The knowledge graph captures various aspects of restaurants including their locations, ratings, amenities, and relationships.

## Overview

The notebook implements a comprehensive restaurant recommendation system using Neo4j graph database. It processes restaurant data to create a rich knowledge graph that captures various relationships between restaurants and their attributes, enabling complex queries and recommendations.

### Key Components:

1. **Data Loading and Processing**
   - Loads restaurant data from CSV files
   - Implements data normalization and text processing utilities
   - Handles Turkish character normalization

2. **Knowledge Graph Structure**
   - **Nodes**: Restaurant, Aspect, Location, Amenity, Occasion, MenuItem
   - **Relationships**:
     - RATED_FOR: Restaurant ratings for different aspects
     - SERVES: Menu items served by restaurants
     - LOCATED_IN: Restaurant locations
     - HAS_AMENITY: Restaurant amenities
     - SUITS_OCCASION: Suitable occasions for restaurants

3. **Features**
   - Sentiment analysis for restaurant aspects
   - Menu item detection and classification
   - Location-based querying
   - Multi-criteria restaurant search
   - Comprehensive rating system

4. **Query Capabilities**
   - Location-based searches (city/district)
   - Menu-based filtering
   - Amenity-based filtering
   - Occasion-based recommendations
   - Rating-based sorting and filtering

### Technical Implementation:

- **Data Processing**:
  - JSON parsing and normalization
  - Text normalization (Turkish-language aware)
  - Aspect mapping and categorization
  - Sentiment analysis using keyword sets

- **Neo4j Integration**:
  - Custom Neo4j knowledge graph class
  - Efficient batch import processes
  - Constraint and index management
  - Complex relationship queries

- **Search Interface**:
  - Natural language query parsing
  - Multi-criteria search support
  - Formatted result presentation
  - Interactive chatbot interface

## Cell 1: Data Loading and Initial Setup


In [ ]:
import pandas as pd
import json
import re
import datetime

data = pd.read_csv('/Users/Serra/Desktop/bitirme/kullanılan csvler/Son_Data_Score.csv')

In [ ]:
data.head()

,Mekan_Adı,Ilce,Il,Total_Weighted_Score,Tat_Score,Hizmet_Score,Ortam_Score,Fiyat-Performans_Score,Menü Çeşitliliği_Score,Temizlik_Score,Özet,Entity
0,01_adana_durumluk,Etimesgut,Ankara,0.028966,0.755000,0.225000,0.850000,0.628571,NaN,1.000000,"Restoran, özellikle Adana dürüm ve kebaplarıyl...","{""restaurant"": ""01_adana_durumluk"", ""language""..."
1,01_adanali_ismail_ustanin_yeri,Çankaya,Ankara,0.160363,0.748000,0.492188,0.718000,0.236667,0.392105,0.470000,"Bu restoran, özellikle **lezzetli Adana kebabı...","{""restaurant"": ""01_adanali_ismail_ustanin_yeri..."
2,06_ankara_kokorec,Pursaklar,Ankara,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,"Bu restoran, özellikle kokoreç konusunda öne ç...","{""restaurant"": ""06_ankara_kokorec"", ""language""..."
3,100_more_alsancak,Konak,İzmir,0.183239,0.764706,0.730000,0.851724,0.296875,0.700000,0.955556,"%100&MORE Alsancak, İzmir'in popüler mekanları...","{\n ""restaurant"": ""100_more_alsancak"",\n ""la..."
4,10_numara_restaurant,Çatalca,İstanbul,0.135506,0.827778,0.821739,0.864000,0.559091,0.960000,1.000000,"Restoran, samimi ve sıcak bir ortamıyla öne çı...","{\n ""restaurant"": ""10_numara_restaurant"",\n ..."


## Utility Functions and Data Processing Helpers


In [ ]:
# ========= 1) Yardımcılar =========
import re, json, ast, datetime
import pandas as pd
import unicodedata

def now_utc_iso():
    return datetime.datetime.now(datetime.timezone.utc).isoformat()

def norm(s: str) -> str:
    """Normalize plain strings, Türkçe karakter uyumlu"""
    if s is None:
        return ""
    s = str(s).strip()
    s = unicodedata.normalize("NFC", s)  # Unicode normalize
    # Türkçe özel case map
    tr_map = str.maketrans({"I": "ı", "İ": "i"})
    s = s.translate(tr_map).casefold()
    return s

def normalize_surface(s: str) -> str:
    """Normalize for surface names (menu, aspect), Türkçe karakter uyumlu"""
    s = (s or "").strip()
    s = unicodedata.normalize("NFC", s)
    tr_map = str.maketrans({"I": "ı", "İ": "i"})
    s = s.translate(tr_map).casefold()
    s = re.sub(r"[^\w\sçğıöşü-]+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def safe_parse_summary(raw):
    if raw is None or (isinstance(raw, float) and pd.isna(raw)):
        return None
    if isinstance(raw, dict):
        return raw
    if isinstance(raw, (bytes, bytearray)):
        raw = raw.decode("utf-8", "ignore")
    s = str(raw).strip()
    if not s:
        return None
    s = s.lstrip("\ufeff").replace("“","\"").replace("”","\"").replace("’","'").replace("‘","'")
    try:
        return json.loads(s)
    except:
        pass
    if "{" in s and "}" in s:
        core = s[s.find("{"): s.rfind("}")+1]
        try:
            return json.loads(core)
        except:
            core2 = re.sub(r",\s*([}\]])", r"\1", core)
            try:
                return json.loads(core2)
            except:
                pass
    try:
        obj = ast.literal_eval(s)
        if isinstance(obj, dict):
            return obj
    except:
        pass
    return None

# ========= 2) Sözlükler / Haritalar =========
ASPECT_MAP = {
  "Tat":"TASTE","Hizmet":"SERVICE","Ortam":"AMBIENCE",
  "Fiyat-Performans":"VALUE","Menü Çeşitliliği":"MENU_VARIETY","Temizlik":"CLEANLINESS"
}

NEG_HINTS = {"değil","yetersiz","küçük","pahalı","kirli","eksik","sorun", "sorunlu","sıkıntı"}
POS_HINTS = {"lezzetli","doyurucu","uygun fiyat","güler yüzlü","temiz"}

AMENITIES = {"otopark","çocuk parkı","bahçe","deniz manzarası","canlı müzik","alkolsüz","teras"}
OCCASIONS = {"romantik","aile","sakin","doğum günü","iş yemeği", "eğlenceli"}

# Menü isimleri (çekirdek; kendi KB’inle genişlet)
MENU_LEX = {
    "adana dürüm","kebap","lahmacun","pide","köfte","döner","balık","çorba","salata",
    "meze","tatlı","baklava","künefe","sushi","ramen","pizza","hamburger","burger",
    "biftek","mantı","falafel","humus","shawarma","taco","burrito","curry",
    "penne","spaghetti","risotto","gnocchi","türk mutfağı", "geleneksel türk mutfağı"
}
MENU_PAT = re.compile(
    r"\b(kebap|dürüm|döner|pide|lahmacun|köfte|mantı|balık|çorba|salata|meze|tatlı|"
    r"baklava|künefe|sushi|ramen|pizza|hamburger|burger|biftek|falafel|humus|shawarma|"
    r"taco|burrito|curry|penne|spaghetti|risotto|gnocchi)s?\b",
    flags=re.IGNORECASE
)

def sentiment_from(polarity, spans):
    text = " ".join(spans or []).lower()
    if any(w in text for w in NEG_HINTS): return "negative"
    if any(w in text for w in POS_HINTS): return "positive"
    if polarity == "+": return "positive"
    if polarity == "-": return "negative"
    return "neutral"

def is_menu_item(term: str) -> bool:
    t = normalize_surface(term)
    return (t in MENU_LEX) or bool(MENU_PAT.search(t))

REL_LOCAL_MAP = {
    "co-occur": "CO_OCCUR",
    "co_occur": "CO_OCCUR",
    "contrast": "CONTRASTS",
    "contrasts": "CONTRASTS",
    "boosts": "BOOSTS",
    "implies": "IMPLIES",
    "affects_negatively": "AFFECTS_NEGATIVELY"
}

## Knowledge Graph Edge Builder
Bu cell'de restaurant verilerinden knowledge graph edge'leri oluşturan ana fonksiyon tanımlanır:
- RATED_FOR: Restaurant'ların aspect'lere göre değerlendirmeleri
- SERVES: Restaurant'ların servis ettiği menü itemları
- HAS_AMENITY: Restaurant'ların sahip olduğu imkanlar (otopark, bahçe, vb.)
- SUITS_OCCASION: Restaurant'ların uygun olduğu durumlar (romantik, aile, vb.)
- LOCATED_IN: Restaurant'ların konum bilgileri (il/ilçe)


In [ ]:
def build_edges(doc, source="llm_entity_json", il=None, ilce=None):
    rest = f"restaurant:{norm(doc.get('restaurant'))}"
    edges = []
    entities = doc.get("entities") or {}

    for k, items in entities.items():
        group = ASPECT_MAP.get(k)
        if not group:
            continue
        for it in items:
            name = (it.get("normalize_to") or it.get("name") or "").strip()
            if not name:
                continue
            spans = it.get("evidence_spans", []) or []
            sent = sentiment_from(it.get("polarity"), spans)
            sal = float(it.get("salience", 0.5))
            conf = float(it.get("confidence", 0.5))
            score = round(0.5*sal + 0.5*conf, 3)

            if group == "TASTE":
                # Menü ise: SERVES yaz, RATED_FOR yazma
                if is_menu_item(name):
                    edges.append({
                        "head": rest, "relation": "SERVES",
                        "tail": f"menu:{normalize_surface(name)}",
                        "props": {
                            "confidence": min(1.0, conf + 0.1),
                            "evidence_spans": spans, "source": source, "updated_at": now_utc_iso()
                        }
                    })
                    continue
                # Menü değilse gerçek TASTE aspekti → RATED_FOR
                edges.append({
                    "head": rest, "relation": "RATED_FOR",
                    "tail": f"aspect:{normalize_surface(name)}",
                    "props": {
                        "dimension": "TASTE", "sentiment": sent, "score": score,
                        "confidence": conf, "evidence_spans": spans, "votes": 1,
                        "source": source, "updated_at": now_utc_iso()
                    }
                })
                continue

            # Diğer gruplar: RATED_FOR
            edges.append({
                "head": rest, "relation": "RATED_FOR",
                "tail": f"aspect:{normalize_surface(name)}",
                "props": {
                    "dimension": group, "sentiment": sent, "score": score,
                    "confidence": conf, "evidence_spans": spans, "votes": 1,
                    "source": source, "updated_at": now_utc_iso()
                }
            })

    # Amenity & Occasion (span araması)
    all_text = " ".join([
        " ".join(e.get("evidence_spans", []) or [])
        for arr in entities.values() for e in arr
    ]).lower()

    for amen in AMENITIES:
        if amen in all_text:
            edges.append({
                "head": rest, "relation": "HAS_AMENITY",
                "tail": f"amenity:{amen}",
                "props": {
                    "availability": "true", "confidence": 0.8,
                    "evidence_spans": [amen], "source": source, "updated_at": now_utc_iso()
                }
            })

    for occ in OCCASIONS:
        if occ in all_text:
            edges.append({
                "head": rest, "relation": "SUITS_OCCASION",
                "tail": f"occasion:{occ}",
                "props": {
                    "strength": "medium", "confidence": 0.75,
                    "evidence_spans": [occ], "source": source, "updated_at": now_utc_iso()
                }
            })

    # LOCATED_IN: İl / İlçe kolonlarından
    if ilce and str(ilce).strip():
        district_name = norm(str(ilce))
        edges.append({
            "head": rest, "relation": "LOCATED_IN",
            "tail": f"location:{district_name}",
            "props": {
                "level": "district",
                "name": district_name,   # 🔹 name eklendi
                "confidence": 1.0,
                "evidence_spans": [],
                "source": source,
                "updated_at": now_utc_iso()
            }
        })
    if il and str(il).strip():
        city_name = norm(str(il))
        edges.append({
            "head": rest, "relation": "LOCATED_IN",
            "tail": f"location:{city_name}",
            "props": {
                "level": "city",
                "name": city_name,       # 🔹 name eklendi
                "confidence": 1.0,
                "evidence_spans": [],
                "source": source,
                "updated_at": now_utc_iso()
            }
        })

            # 3.5 NEW: relations_local → edges
    for lr in (doc.get("relations_local") or []):
        rtype = REL_LOCAL_MAP.get(str(lr.get("type", "")).lower())
        if not rtype:
            continue
        a = lr.get("a"); b = lr.get("b")
        if not a or not b:
            continue
        edges.append({
            "head": f"aspect:{normalize_surface(a)}",
            "relation": rtype,
            "tail": f"aspect:{normalize_surface(b)}",
            "props": {
                "strength": lr.get("strength"),
                "source": source, "updated_at": now_utc_iso()
            }
        })

    return edges

## Edge Generation and DataFrame Creation
Bu cell'de tüm DataFrame satırları üzerinde döngü yapılarak edge'ler oluşturulur:
- Her satırdaki JSON verisi parse edilir
- build_edges fonksiyonu çağırılarak ilişkiler çıkarılır
- Tüm edge'ler tek bir DataFrame'de toplanır
- LOCATED_IN edge'lerinin sayısı kontrol edilir (498 adet)


In [ ]:
# ========= 4) DataFrame üzerinde çalıştır =========
city_col     = "Il"    # İl için kolon adı
district_col = "Ilce"  # İlçe için kolon adı

all_edges = []
for idx, row in data.iterrows():
    try:
        doc = safe_parse_summary(row.get("Entity"))
        if doc is None:
            continue

        il   = row.get(city_col)      # "Sehir"
        ilce = row.get(district_col)  # "Ilce"

        edges = build_edges(doc, source=f"row_{idx}", il=il, ilce=ilce)
        for e in edges:
            e["doc_index"] = idx
            all_edges.append(e)
    except Exception as ex:
        print(f"[ERROR] row {idx}: {ex}")

edges_df = pd.DataFrame(all_edges)

# Hızlı kontrol:
print("LOCATED_IN edge sayısı:", (edges_df["relation"] == "LOCATED_IN").sum())
print(edges_df[edges_df["relation"] == "LOCATED_IN"].head(20))

LOCATED_IN edge sayısı: 4670
                                                  head    relation  \
12                        restaurant:01_adana_durumluk  LOCATED_IN   
13                        restaurant:01_adana_durumluk  LOCATED_IN   
32           restaurant:01_adanali_ismail_ustanin_yeri  LOCATED_IN   
33           restaurant:01_adanali_ismail_ustanin_yeri  LOCATED_IN   
48                        restaurant:06_ankara_kokorec  LOCATED_IN   
49                        restaurant:06_ankara_kokorec  LOCATED_IN   
69                        restaurant:100_more_alsancak  LOCATED_IN   
70                        restaurant:100_more_alsancak  LOCATED_IN   
86                     restaurant:10_numara_restaurant  LOCATED_IN   
87                     restaurant:10_numara_restaurant  LOCATED_IN   
115                   restaurant:1453_osmanli_bostanli  LOCATED_IN   
116                   restaurant:1453_osmanli_bostanli  LOCATED_IN   
141  restaurant:16_roof_swissotel_the_bosphorus_ist...  LOCAT

In [ ]:
# `props` kolonunu normalleştir
props_normalized = pd.json_normalize(edges_df['props'])

# Normalleştirilmiş DataFrame ile orijinal DataFrame'i birleştir
edges_df_normalized = pd.concat([edges_df, props_normalized], axis=1)

# `level` ve `name` kolonlarında benzersiz değerleri al
distinct_values = edges_df_normalized[['level', 'name']].drop_duplicates()
# Sonuçları yazdır
distinct_values[distinct_values["level"] == "city"]

,level,name
13,city,ankara
70,city,izmir
87,city,istanbul


In [ ]:
edges_df["relation"].value_counts()

relation
RATED_FOR         36709
LOCATED_IN         4670
CO_OCCUR           3490
SERVES             2961
SUITS_OCCASION      622
HAS_AMENITY         442
CONTRASTS           153
IMPLIES               4
Name: count, dtype: int64

## Neo4j Package Installation

In [ ]:
# Install required packages for Neo4j
# Run this cell first to install dependencies
%pip install neo4j pandas

Note: you may need to restart the kernel to use updated packages.


## Neo4j Knowledge Graph Class Definition

**Node türleri:** Restaurant, Aspect, Location, Amenity, Occasion, MenuItem
**Relationship türleri:** RATED_FOR, SERVES, LOCATED_IN, HAS_AMENITY, SUITS_OCCASION


In [ ]:
# ========= 5) Neo4j Knowledge Graph Builder =========
from neo4j import GraphDatabase
import logging

class Neo4jKnowledgeGraph:
    def __init__(self, uri="bolt://localhost:7687", user="neo4j", password="password"):
        """
        Initialize Neo4j connection

        Default connection assumes local Neo4j instance:
        - URI: bolt://localhost:7687
        - User: neo4j
        - Password: password (change this!)

        Make sure Neo4j is running before using this class.
        """
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def clear_database(self):
        """Clear all nodes and relationships - USE WITH CAUTION!"""
        with self.driver.session() as session:
            session.run("MATCH (n) DETACH DELETE n")
            print("Database cleared!")

    def create_constraints_and_indexes(self):
        """Create constraints and indexes for better performance"""
        with self.driver.session() as session:
            constraints_and_indexes = [
                # Constraints (ensure uniqueness)
                "CREATE CONSTRAINT restaurant_id IF NOT EXISTS FOR (r:Restaurant) REQUIRE r.id IS UNIQUE",
                "CREATE CONSTRAINT aspect_id IF NOT EXISTS FOR (a:Aspect) REQUIRE a.id IS UNIQUE",
                "CREATE CONSTRAINT location_id IF NOT EXISTS FOR (l:Location) REQUIRE l.id IS UNIQUE",
                "CREATE CONSTRAINT amenity_id IF NOT EXISTS FOR (am:Amenity) REQUIRE am.id IS UNIQUE",
                "CREATE CONSTRAINT occasion_id IF NOT EXISTS FOR (o:Occasion) REQUIRE o.id IS UNIQUE",
                "CREATE CONSTRAINT menu_id IF NOT EXISTS FOR (m:MenuItem) REQUIRE m.id IS UNIQUE",

                # Indexes for faster queries
                "CREATE INDEX restaurant_name IF NOT EXISTS FOR (r:Restaurant) ON (r.name)",
                "CREATE INDEX location_name IF NOT EXISTS FOR (l:Location) ON (l.name)",
            ]

            for cmd in constraints_and_indexes:
                try:
                    session.run(cmd)
                    print(f"✓ {cmd.split()[1]} created")
                except Exception as e:
                    print(f"⚠ {cmd.split()[1]} already exists or error: {e}")

    def extract_node_info(self, node_id):
        """Extract node type and properties from node_id like 'restaurant:name' or 'aspect:taste'"""
        if ":" not in node_id:
            return "Unknown", {"id": node_id, "name": node_id}

        node_type, name = node_id.split(":", 1)

        # Map node types to Neo4j labels
        label_map = {
            "restaurant": "Restaurant",
            "aspect": "Aspect",
            "location": "Location",
            "amenity": "Amenity",
            "occasion": "Occasion",
            "menu": "MenuItem"
        }

        label = label_map.get(node_type, "Entity")
        props = {"id": node_id, "name": name, "type": node_type}

        return label, props

    def get_stats(self):
        """Get basic statistics about the knowledge graph"""
        with self.driver.session() as session:
            # Count nodes by label
            node_counts = {}
            labels_result = session.run("CALL db.labels()")
            for record in labels_result:
                label = record[0]
                count_result = session.run(f"MATCH (n:{label}) RETURN count(n) as count")
                node_counts[label] = count_result.single()['count']

            # Count relationships by type
            rel_counts = {}
            rel_types_result = session.run("CALL db.relationshipTypes()")
            for record in rel_types_result:
                rel_type = record[0]
                count_result = session.run(f"MATCH ()-[r:{rel_type}]->() RETURN count(r) as count")
                rel_counts[rel_type] = count_result.single()['count']

            return {
                'nodes': node_counts,
                'relationships': rel_counts,
                'total_nodes': sum(node_counts.values()),
                'total_relationships': sum(rel_counts.values())
            }

    def create_node(self, node_id, additional_props=None):
        """Create a single node in Neo4j"""
        label, props = self.extract_node_info(node_id)

        if additional_props:
            props.update(additional_props)

        # Şehir isimlerini normalize et
        if 'name' in props:
            props['name'] = norm(props['name'])

        with self.driver.session() as session:
            query = f"""
            MERGE (n:{label} {{id: $id}})
            SET n += $props
            RETURN n
            """
            session.run(query, id=node_id, props=props)


    def create_relationship(self, head_id, relation, tail_id, props=None):
        """Create a relationship between two nodes"""
        # Ensure both nodes exist first
        self.create_node(head_id)
        self.create_node(tail_id)

        head_label, _ = self.extract_node_info(head_id)
        tail_label, _ = self.extract_node_info(tail_id)

        props = props or {}

        with self.driver.session() as session:
            query = f"""
            MATCH (h:{head_label} {{id: $head_id}})
            MATCH (t:{tail_label} {{id: $tail_id}})
            MERGE (h)-[r:{relation}]->(t)
            SET r += $props
            RETURN r
            """
            session.run(query, head_id=head_id, tail_id=tail_id, props=props)

    def bulk_import_edges(self, edges_df, batch_size=1000):
        """Efficiently import all edges from the DataFrame"""
        print(f"Importing {len(edges_df)} edges in batches of {batch_size}...")

        # First pass: Create all unique nodes
        unique_nodes = set()
        for _, row in edges_df.iterrows():
            unique_nodes.add(row['head'])
            unique_nodes.add(row['tail'])

        print(f"Creating {len(unique_nodes)} unique nodes...")

        with self.driver.session() as session:
            for i, node_id in enumerate(unique_nodes):
                if i % 100 == 0:
                    print(f"  Nodes created: {i}/{len(unique_nodes)}")

                label, props = self.extract_node_info(node_id)
                query = f"""
                MERGE (n:{label} {{id: $id}})
                SET n += $props
                """
                session.run(query, id=node_id, props=props)

        print("✓ All nodes created!")

        # Second pass: Create relationships in batches
        print("Creating relationships...")

        for i in range(0, len(edges_df), batch_size):
            batch = edges_df.iloc[i:i+batch_size]
            print(f"  Processing batch {i//batch_size + 1}/{(len(edges_df)-1)//batch_size + 1}")

            with self.driver.session() as session:
                for _, row in batch.iterrows():
                    head_id = row['head']
                    tail_id = row['tail']
                    relation = row['relation']
                    props = row.get('props', {})

                    # Add doc_index if available
                    if 'doc_index' in row:
                        props['doc_index'] = row['doc_index']

                    head_label, _ = self.extract_node_info(head_id)
                    tail_label, _ = self.extract_node_info(tail_id)

                    query = f"""
                    MATCH (h:{head_label} {{id: $head_id}})
                    MATCH (t:{tail_label} {{id: $tail_id}})
                    MERGE (h)-[r:{relation}]->(t)
                    SET r += $props
                    """
                    session.run(query, head_id=head_id, tail_id=tail_id, props=props)

        print("✓ All relationships created!")


## Neo4j Connection Test

In [ ]:
# Initialize the knowledge graph
kg = Neo4jKnowledgeGraph(
    uri="neo4j://127.0.0.1:7687",
    user="neo4j",
    password="12345678"  # ← bunu güçlü bir şifre ile değiştir
)

try:
    # Test connection
    print("Testing Neo4j connection...")
    stats = kg.get_stats()
    print("✓ Connected to Neo4j successfully!")
    print(f"Current database has {stats['total_nodes']} nodes and {stats['total_relationships']} relationships")

except Exception as e:
    print(f"❌ Connection failed: {e}")
    print("\nMake sure:")
    print("1. Neo4j is running")
    print("2. The password is correct")
    print("3. The URI is correct (default: bolt://localhost:7687)")
    kg = None


Testing Neo4j connection...
✓ Connected to Neo4j successfully!
Current database has 13386 nodes and 50183 relationships


## Knowledge Graph Creation and Data Import

**Adımlar:**
1. Database schema oluşturma (constraints ve indexes)
2. 4522 edge'i 500'lük batch'ler halinde import etme
3. 2140 unique node oluşturma
4. 4479 relationship oluşturma

**Final İstatistikler:**
- Restaurant: 249, Aspect: 1704, Location: 24, Amenity: 6, Occasion: 5, MenuItem: 152
- RATED_FOR: 3564, SERVES: 312, LOCATED_IN: 496, HAS_AMENITY: 44, SUITS_OCCASION: 63


In [ ]:
# ========= 7) Import Data into Neo4j =========

if kg is not None:
    print("Setting up database schema...")

    # Create constraints and indexes
    kg.create_constraints_and_indexes()

    print(f"\nImporting {len(edges_df)} edges into Neo4j...")
    print("This may take a few minutes...")

    # Import all edges (this creates nodes and relationships)
    kg.bulk_import_edges(edges_df, batch_size=500)

    print("\n🎉 Knowledge graph created successfully!")

    # Get final statistics
    final_stats = kg.get_stats()
    print(f"\nFinal Statistics:")
    print(f"📊 Total Nodes: {final_stats['total_nodes']}")
    print(f"🔗 Total Relationships: {final_stats['total_relationships']}")
    print(f"\nNodes by type:")
    for label, count in final_stats['nodes'].items():
        print(f"  {label}: {count}")
    print(f"\nRelationships by type:")
    for rel_type, count in final_stats['relationships'].items():
        print(f"  {rel_type}: {count}")

else:
    print("❌ Skipping import - no Neo4j connection")

Setting up database schema...
✓ CONSTRAINT created
✓ CONSTRAINT created
✓ CONSTRAINT created
✓ CONSTRAINT created
✓ CONSTRAINT created
✓ CONSTRAINT created
✓ INDEX created
✓ INDEX created

Importing 49051 edges into Neo4j...
This may take a few minutes...
Importing 49051 edges in batches of 500...
Creating 13034 unique nodes...
  Nodes created: 0/13034
  Nodes created: 100/13034
  Nodes created: 200/13034
  Nodes created: 300/13034
  Nodes created: 400/13034
  Nodes created: 500/13034
  Nodes created: 600/13034
  Nodes created: 700/13034
  Nodes created: 800/13034
  Nodes created: 900/13034
  Nodes created: 1000/13034
  Nodes created: 1100/13034
  Nodes created: 1200/13034
  Nodes created: 1300/13034
  Nodes created: 1400/13034
  Nodes created: 1500/13034
  Nodes created: 1600/13034
  Nodes created: 1700/13034
  Nodes created: 1800/13034
  Nodes created: 1900/13034
  Nodes created: 2000/13034
  Nodes created: 2100/13034
  Nodes created: 2200/13034
  Nodes created: 2300/13034
  Nodes cr

## Validation Queries and Knowledge Graph Testing



In [ ]:
def validate_relationships(kg):
    """Neo4j Knowledge Graph üzerinde ilişkilerin doğruluğunu test eder"""
    if kg is None:
        print("❌ Neo4j bağlantısı yok")
        return

    queries = [
        {
            "name": "RATED_FOR Relationship",
            "query": """
            MATCH (r:Restaurant)-[rating:RATED_FOR]->(a:Aspect)
            RETURN r.name as restoran, a.name as aspekt, rating.sentiment as sentiment, rating.score as puan
            LIMIT 10
            """,
            "description": "RATED_FOR ilişkisi, restoranların değerlendirmelerinin doğru bağlandığını kontrol eder"
        },

        {
            "name": "LOCATED_IN Relationship",
            "query": """
            MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
            RETURN r.name as restoran, l.name as lokasyon
            LIMIT 10
            """,
            "description": "LOCATED_IN ilişkisi, restoranların doğru lokasyonlarla eşlendiğini kontrol eder"
        },

        {
            "name": "SERVES Relationship",
            "query": """
            MATCH (r:Restaurant)-[:SERVES]->(m:MenuItem)
            RETURN r.name as restoran, m.name as menu_item
            LIMIT 10
            """,
            "description": "SERVES ilişkisi, restoranların menü öğeleriyle doğru bağlandığını kontrol eder"
        },

        {
            "name": "SUITS_OCCASION Relationship",
            "query": """
            MATCH (r:Restaurant)-[:SUITS_OCCASION]->(o:Occasion)
            RETURN r.name as restoran, o.name as occasion
            LIMIT 10
            """,
            "description": "SUITS_OCCASION ilişkisi, restoranların doğru özel günlere (örneğin, romantik, aile) eşlendiğini kontrol eder"
        },

        {
            "name": "HAS_AMENITY Relationship",
            "query": """
            MATCH (r:Restaurant)-[:HAS_AMENITY]->(am:Amenity)
            RETURN r.name as restoran, am.name as amenity
            LIMIT 10
            """,
            "description": "HAS_AMENITY ilişkisi, restoranların sahip olduğu imkanların doğru bağlandığını kontrol eder"
        }
    ]

    print("🔍 İlişkiler doğrulama sorguları çalıştırılıyor...\n")

    with kg.driver.session() as session:
        for i, q in enumerate(queries, 1):
            print(f"{i}. {q['name']}")
            print(f"   {q['description']}")

            try:
                result = session.run(q['query'])
                records = list(result)

                if records:
                    print(f"   ✓ {len(records)} sonuç bulundu:")
                    for j, record in enumerate(records[:3]):  # İlk 3 sonucu göster
                        print(f"     {j+1}. {dict(record)}")
                    if len(records) > 3:
                        print(f"     ... ve {len(records) - 3} tane daha")
                else:
                    print("   ⚠ Sonuç bulunamadı")

            except Exception as e:
                print(f"   ❌ Sorgu hatası: {e}")

            print()

# Çalıştır
validate_relationships(kg)


🔍 İlişkiler doğrulama sorguları çalıştırılıyor...

1. RATED_FOR Relationship
   RATED_FOR ilişkisi, restoranların değerlendirmelerinin doğru bağlandığını kontrol eder
   ✓ 10 sonuç bulundu:
     1. {'restoran': 'beyzade_cafe_elmadag', 'aspekt': 'kahve kalitesi', 'sentiment': 'negative', 'puan': 0.825}
     2. {'restoran': 'beyzade_cafe_elmadag', 'aspekt': 'ferah', 'sentiment': 'positive', 'puan': 0.94}
     3. {'restoran': 'beyzade_cafe_elmadag', 'aspekt': 'sipariş bekletme', 'sentiment': 'negative', 'puan': 0.775}
     ... ve 7 tane daha

2. LOCATED_IN Relationship
   LOCATED_IN ilişkisi, restoranların doğru lokasyonlarla eşlendiğini kontrol eder
   ✓ 10 sonuç bulundu:
     1. {'restoran': 'cagla_cafe', 'lokasyon': 'kahramankazan'}
     2. {'restoran': 'altin_yaprak_doner', 'lokasyon': 'kahramankazan'}
     3. {'restoran': '19_pide_kebap_ustanin_yeri', 'lokasyon': 'kahramankazan'}
     ... ve 7 tane daha

3. SERVES Relationship
   SERVES ilişkisi, restoranların menü öğeleriyle doğru b

## Kompleks Çoklu Relation Sorguları
Bu cell'de "Ankara Beypazarı'nda çocuk bahçesi olan kebapçı" tarzı kompleks sorgular için geliştirilmiş fonksiyonlar bulunur:

**Yeni özellikler:**
- `search_restaurants_multi_criteria()`: Şehir + ilçe + menü + amenity kombinasyonları
- `find_restaurants_with_all_criteria()`: Dictionary tabanlı esnek arama

In [ ]:
def chatbot_query_restaurants(kg, criteria, limit=3):
    """
    Chatbot için restoran arama fonksiyonu.
    criteria dict örneği:
    {
        "city": "ankara",
        "district": "beypazarı",
        "menu": "kebap",
        "amenities": ["çocuk parkı"],
        "occasion": "romantik",
        "min_score": 0.7,
        "aspects": ["temiz", "cana yakın personel"]
    }
    """
    if kg is None:
        return [{"error": "Neo4j bağlantısı yok"}]

    with kg.driver.session() as session:
        match_clauses = ["(r:Restaurant)"]
        conditions = []
        params = {"limit": limit}

        # Şehir
        if "city" in criteria:
            match_clauses.append("(r)-[:LOCATED_IN]->(city:Location)")
            conditions.append("toLower(city.name) = $city")
            params["city"] = criteria["city"].lower()

        # İlçe
        if "district" in criteria:
            match_clauses.append("(r)-[:LOCATED_IN]->(district:Location)")
            conditions.append("toLower(district.name) = $district")
            params["district"] = criteria["district"].lower()

        # Menü
        if "menu" in criteria:
            match_clauses.append("(r)-[:SERVES]->(m:MenuItem)")
            conditions.append("toLower(m.name) CONTAINS $menu")
            params["menu"] = criteria["menu"].lower()

        # Amenity
        if "amenities" in criteria:
            for i, amenity in enumerate(criteria["amenities"]):
                match_clauses.append(f"(r)-[:HAS_AMENITY]->(a{i}:Amenity)")
                conditions.append(f"toLower(a{i}.name) CONTAINS $amenity{i}")
                params[f"amenity{i}"] = amenity.lower()

        # Occasion
        if "occasion" in criteria:
            match_clauses.append("(r)-[:SUITS_OCCASION]->(o:Occasion)")
            conditions.append("toLower(o.name) = $occasion")
            params["occasion"] = criteria["occasion"].lower()

        # Aspect (Yeni eklenen kısım)
        if "aspects" in criteria:
            for i, aspect in enumerate(criteria["aspects"]):
                match_clauses.append(f"(r)-[:RATED_FOR]->(asp{i}:Aspect)")
                conditions.append(f"toLower(asp{i}.name) CONTAINS $aspect{i}")
                params[f"aspect{i}"] = aspect.lower()

        # Min score
        if "min_score" in criteria:
            match_clauses.append("(r)-[rating:RATED_FOR]->(asp:Aspect)")
            conditions.append("rating.score >= $min_score AND rating.sentiment = 'positive'")
            params["min_score"] = criteria["min_score"]

        match_clause = "MATCH " + ", ".join(match_clauses)
        where_clause = "WHERE " + " AND ".join(conditions) if conditions else ""

        query = f"""
        {match_clause}
        {where_clause}
        OPTIONAL MATCH (r)-[rt:RATED_FOR]->(asp:Aspect)
        OPTIONAL MATCH (r)-[:HAS_AMENITY]->(am:Amenity)
        OPTIONAL MATCH (r)-[:SERVES]->(m:MenuItem)
        OPTIONAL MATCH (r)-[:SUITS_OCCASION]->(o:Occasion)
        WHERE rt.sentiment = 'positive'
        WITH r,
             avg(rt.score) as avg_score,
             collect(DISTINCT asp.name) as aspects,
             collect(DISTINCT am.name) as amenities,
             collect(DISTINCT m.name) as menu_items,
             collect(DISTINCT o.name) as occasions
        RETURN r.name as restoran,
               round(avg_score,2) as ortalama_puan,
               aspects, amenities, menu_items, occasions
        ORDER BY ortalama_puan DESC
        LIMIT $limit
        """

        result = session.run(query, params)
        return [dict(record) for record in result]


In [ ]:
# Test 1: Şehir ve Menü Öğesi ile Restoran Arama
criteria = {
    "city": "ankara",
    "menu": "kebap",
    "limit": 3  # LIMIT'i 5'ten 3'e düşürdük
}
result = chatbot_query_restaurants(kg, criteria)
print(result)

# Test 2: İlçe ve Olanaklar ile Arama
criteria = {
    "district": "beypazarı",
    "amenities": ["çocuk parkı"],
    "limit": 3
}
result = chatbot_query_restaurants(kg, criteria)
print(result)

# Test 3: Özel Gün ve Tat Puanına Göre Arama
criteria = {
    "city": "izmir",
    "occasion": "romantik",
    "min_score": 0.7,
    "limit": 3
}
result = chatbot_query_restaurants(kg, criteria)
print(result)

# Test 4: Aspektler ile Arama
criteria = {
    "aspects": ["temiz", "cana yakın personel"],
    "min_score": 0.7,
    "limit": 3
}
result = chatbot_query_restaurants(kg, criteria)
print(result)


[{'restoran': 'oz_dedem_pide_kebap_salonu_selahattin_ustanin_yeri_sincanda_baska_subem_yoktursincan_pide_kebap', 'ortalama_puan': 0.89, 'aspects': ['yüksek fiyat', 'hızlı servis', 'ilgili personel', 'güler yüzlü personel', 'samimi ortam', 'makul fiyat', 'hijyen', 'yardımsever personel', 'usta ustalığı'], 'amenities': [], 'menu_items': ['kebap lezzeti'], 'occasions': ['aile']}, {'restoran': 'baglum_halil_ibrahim_sofrasi', 'ortalama_puan': 0.88, 'aspects': ['mantar', 'kalite', 'uygun fiyat', 'fiyat-performans', 'temizlik', 'ikramlar', 'lezzet', 'güler yüzlü', 'gecikme', 'ilgili', 'misafirperver', 'yanlış ürün', 'boğucu', 'bol ikram', 'hijyen'], 'amenities': [], 'menu_items': ['kebap'], 'occasions': []}, {'restoran': 'mis_gaziantep_pide_ve_kebap_salonu', 'ortalama_puan': 0.87, 'aspects': ['lezzetli yemek', 'temiz', 'mutfak hijyeni', 'beyti', 'uygun fiyat', 'yavaş servis', 'eksik sipariş', 'adana kebabı', 'yanlış sipariş', 'istikrarsız hizmet kalitesi', 'nezih ortam', 'doyurucu porsiyon', 

In [ ]:
def format_chatbot_response(results):
    if not results:
        return "⚠ Hiç uygun mekan bulunamadı."

    cevap = ["✨ Önerilen mekanlar:"]

    for i, r in enumerate(results, 1):
        # Başlangıç: Restoran adı ve puanı
        line = f"{i}. {r.get('restoran', 'Bilinmeyen Restoran')} (Puan: {r.get('ortalama_puan', 'Bilgi yok')})"

        # Aspects (Öne çıkan özellikler)
        aspects = r.get("aspects", [])
        if aspects:
            line += f" | Öne çıkan: {', '.join(aspects[:3])}"

        # İmkanlar (amenities)
        amenities = r.get("amenities", [])
        if amenities:
            line += f" | İmkanlar: {', '.join(amenities)}"

        # Uygunluk (occasions)
        occasions = r.get("occasions", [])
        if occasions:
            line += f" | Uygun: {', '.join(occasions)}"

        # Menü (menu_items)
        menu_items = r.get("menu_items", [])
        if menu_items:
            line += f" | Menü: {', '.join(menu_items[:3])}"

        # Satıra eklenen restoran bilgilerini cevap listesine ekliyoruz
        cevap.append(line)

    return "\n".join(cevap)



In [ ]:
def get_restaurant_summary(restaurant_name, data):
    # Veriyi filtreleyerek restoranın özetini bulma
    restaurant_summary = data[data['Mekan_Adı'] == restaurant_name]['Özet'].values
    return restaurant_summary[0] if len(restaurant_summary) > 0 else "Özet bulunamadı"

def format_chatbot_response(results, data):
    if not results:
        return "⚠ Hiç uygun mekan bulunamadı."

    cevap = ["✨ Önerilen mekanlar:"]

    for i, r in enumerate(results, 1):
        # Başlangıç: Restoran adı ve puanı
        line = f"{i}. {r.get('restoran', 'Bilinmeyen Restoran')} (Puan: {r.get('ortalama_puan', 'Bilgi yok')})"

        # Özet bilgisi (data dosyasından alıyoruz)
        restoran_ozet = get_restaurant_summary(r.get('restoran'), data)
        line += f" | Özet: {restoran_ozet}"

        # İmkanlar (amenities)
        amenities = r.get("amenities", [])
        if amenities:
            line += f" | İmkanlar: {', '.join(amenities)}"

        # Uygunluk (occasions)
        occasions = r.get("occasions", [])
        if occasions:
            line += f" | Uygun: {', '.join(occasions)}"

        # Menü (menu_items)
        menu_items = r.get("menu_items", [])
        if menu_items:
            line += f" | Menü: {', '.join(menu_items[:3])}"

        # Satıra eklenen restoran bilgilerini cevap listesine ekliyoruz
        cevap.append(line)

    return "\n".join(cevap)

results = [
    {'restoran': 'öz_dedem_pide_kebap_salonu', 'ortalama_puan': 0.89, 'aspects': ['yüksek fiyat', 'hızlı servis'], 'amenities': [], 'menu_items': ['kebap'], 'occasions': ['aile']},
    {'restoran': 'baglum_halil_ibrahim_sofrasi', 'ortalama_puan': 0.88, 'aspects': ['mantar', 'kalite'], 'amenities': [], 'menu_items': ['kebap'], 'occasions': []},
    {'restoran': 'mis_gaziantep_pide_ve_kebap_salonu', 'ortalama_puan': 0.87, 'aspects': ['lezzetli yemek', 'temiz'], 'amenities': [], 'menu_items': ['beyti'], 'occasions': ['aile']}
]

# Formatlı cevap
formatted_response = format_chatbot_response(results, data)
print(formatted_response)

✨ Önerilen mekanlar:
1. öz_dedem_pide_kebap_salonu (Puan: 0.89) | Özet: Özet bulunamadı | Uygun: aile | Menü: kebap
2. baglum_halil_ibrahim_sofrasi (Puan: 0.88) | Özet: Bu yorumların genelinde restoranın öne çıkan özellikleri şunlar:

**Lezzet:** Yemeklerin kalitesi ve lezzeti sıkça övülüyor. Özellikle pide, lahmacun, kebap çeşitleri ve ikramlar (çiğ köfte, mantar, salata) beğeniliyor. 
**Hizmet Kalitesi:** Çalışanların güler yüzlü, ilgili ve misafirperver olduğu vurgulanıyor. İkramların bol olması da dikkat çekiyor.
**Temizlik ve Hijyen:** Restoranın temizliği ve hijyeni müşteriler tarafından takdir ediliyor. 
**Fiyat-Performans Dengesi:** Yemeklerin lezzeti göz önüne alındığında fiyatların uygun olduğu belirtiliyor.

**Olumsuz Yönler:** Bazı yorumlarda siparişlerde gecikme, yanlış ürün gönderimi ve mekanın biraz boğucu olması gibi eleştiriler yer alıyor. Ancak bu şikayetler genel memnuniyeti gölgelemiyor. 

Sonuç olarak, restoran lezzetli yemekleri, kaliteli hizmeti ve uygun fiyatlar

## Kompleks Sorgu Test Örnekleri


In [ ]:
print("🎯 CHATBOT TABANLI ÇOKLU RELATION TESTLERİ")
print("=" * 60)

# Neo4j bağlantısı var mı kontrol edelim
if not kg:
    print("❌ Neo4j bağlantısı yok!")
else:
    print("✅ Neo4j bağlantısı aktif")

# Test 1: Ankara Beypazarı'nda çocuk parkı olan kebapçı
print("\n🔥 TEST 1: Ankara Beypazarı'nda çocuk parkı olan kebapçı")
criteria1 = {
    "city": "ankara",
    "district": "beypazarı",
    "menu": "kebap",
    "amenities": ["çocuk parkı"]
}
results1 = chatbot_query_restaurants(kg, criteria1, limit=5) if kg else []
if results1:
    print(format_chatbot_response(results1, data))  # Sonuçları formatla ve göster
else:
    print("❌ Bu kriterlere uygun restoran bulunamadı.")
    print("💡 Daha az kriter ile tekrar deneyin.")

# Test 2: Ankara'da bahçeli restoranlar
print("\n🔥 TEST 2: Ankara'da bahçeli restoranlar")
criteria2 = {
    "city": "ankara",
    "amenities": ["bahçe"]
}
results2 = chatbot_query_restaurants(kg, criteria2, limit=5) if kg else []
if results2:
    print(format_chatbot_response(results2, data))  # Sonuçları formatla ve göster
else:
    print("❌ Ankara'da bahçeli restoran bulunamadı.")

# Test 3: İzmir Konak'ta pizza mekanı
print("\n🔥 TEST 3: İzmir Konak'ta pizza mekanı")
criteria3 = {
    "city": "izmir",
    "district": "konak",
    "menu": "pizza"
}
results3 = chatbot_query_restaurants(kg, criteria3, limit=5) if kg else []
if results3:
    print(format_chatbot_response(results3, data))  # Sonuçları formatla ve göster
else:
    print("❌ İzmir Konak'ta pizza mekanı bulunamadı.")

# Test 4: İstanbul'da romantik kebapçı
print("\n🔥 TEST 4: İstanbul'da romantik kebapçı")
criteria4 = {
    "city": "istanbul",
    "menu": "kebap",
    "occasion": "romantik"
}
results4 = chatbot_query_restaurants(kg, criteria4, limit=5) if kg else []
if results4:
    print(format_chatbot_response(results4, data))  # Sonuçları formatla ve göster
else:
    print("❌ İstanbul'da romantik kebapçı bulunamadı.")

# Test 5: İzmir'de yüksek puanlı pizzacılar
print("\n🔥 TEST 5: İzmir'de yüksek puanlı pizzacılar")
criteria5 = {
    "city": "izmir",
    "menu": "pizza",
    "min_score": 0.8
}
results5 = chatbot_query_restaurants(kg, criteria5, limit=5) if kg else []
if results5:
    print(format_chatbot_response(results5, data))  # Sonuçları formatla ve göster
else:
    print("❌ İzmir'de yüksek puanlı pizzacılar bulunamadı.")

# Test 6: İstanbul'da romantik kebapçı ve temiz/cana yakın personel
print("\n🔥 TEST 6: İstanbul'da temiz burgerci")
criteria_aspects = {
    "city": "istanbul",
    "menu": "burger",
    "aspects": ["temiz"]
}
results_aspects = chatbot_query_restaurants(kg, criteria_aspects, limit=5) if kg else []
if results_aspects:
    print(format_chatbot_response(results_aspects, data))  # Sonuçları formatla ve göster
else:
    print("❌ İstanbul'da belirtilen özelliklerde restoran bulunamadı.")
    print("💡 Daha az kriter ile tekrar deneyin.")


🎯 CHATBOT TABANLI ÇOKLU RELATION TESTLERİ
✅ Neo4j bağlantısı aktif

🔥 TEST 1: Ankara Beypazarı'nda çocuk parkı olan kebapçı
❌ Bu kriterlere uygun restoran bulunamadı.
💡 Daha az kriter ile tekrar deneyin.

🔥 TEST 2: Ankara'da bahçeli restoranlar
✨ Önerilen mekanlar:
1. koy_bahcesi (Puan: 0.88) | Özet: Bu yorumların özeti şu şekilde olabilir:

**Köy Bahçesi**, genel olarak **güzel bir mekan** olarak değerlendiriliyor. Özellikle **açık havada kahvaltı yapılabilecek ferah bahçesi, çocuk oyun alanları ve otantik atmosferi** beğeniliyor. Çalışanların çoğu **güler yüzlü ve ilgili** bulunuyor. 

Ancak bazı önemli eksiklikler de vurgulanıyor:

- **Kahvaltılık ürünlerin kalitesi ve çeşidi** yetersiz bulunmuş, özellikle peynirler, reçeller ve domates/salatalık gibi temel malzemelerin lezzetli olmadığı belirtiliyor.
- **Servis hızı yavaş**, siparişler gecikiyor ve bazı ürünler (örneğin gözleme) saatlerce bekletmek zorunda kalınıyor. 
- **Fiyatlar yüksek** bulunuyor, özellikle semaver çayı gibi bas

In [ ]:
def format_chatbot_response_simple(results):
    if not results:
        return "⚠ Hiç uygun mekan bulunamadı."

    cevap = ["✨ Önerilen mekanlar:"]

    # Restoranları numaralandırarak listele
    for i, r in enumerate(results, 1):
        # Restoran adı (Bilinmeyen restoran adını döndürmek yerine sadece adı alıyoruz)
        line = f"{i}. {r.get('restoran', 'Bilinmeyen Restoran')}"

        # Satıra eklenen restoran bilgisini cevap listesine ekliyoruz
        cevap.append(line)

    return "\n".join(cevap)

print("🎯 CHATBOT TABANLI ÇOKLU RELATION TESTLERİ")
print("=" * 60)

# Neo4j bağlantısı var mı kontrol edelim
if not kg:
    print("❌ Neo4j bağlantısı yok!")
else:
    print("✅ Neo4j bağlantısı aktif")

# Test 1: Ankara'da kebap menüsü ve çocuk parkı olan restoranlar
print("\n🔥 TEST 1: Ankara'da kebap menüsü ve çocuk parkı olan restoranlar")
criteria1 = {
    "city": "ankara",
    "menu": "kebap",
    "amenities": ["çocuk parkı"],
    "limit": 3
}
results1 = chatbot_query_restaurants(kg, criteria1, limit=5) if kg else []
if results1:
    print(format_chatbot_response_simple(results1))  # Sonuçları formatla ve sadece isimleri göster
else:
    print("❌ Bu kriterlere uygun restoran bulunamadı.")
    print("💡 Daha az kriter ile tekrar deneyin.")

# Test 2: İstanbul'da romantik ve yüksek puanlı kebap restoranları
print("\n🔥 TEST 2: İstanbul'da romantik ve yüksek puanlı kebap restoranları")
criteria2 = {
    "city": "istanbul",
    "menu": "kebap",
    "occasion": "romantik",
    "min_score": 0.8,
    "limit": 3
}
results2 = chatbot_query_restaurants(kg, criteria2, limit=5) if kg else []
if results2:
    print(format_chatbot_response_simple(results2))  # Sonuçları formatla ve sadece isimleri göster
else:
    print("❌ İstanbul'da romantik ve yüksek puanlı kebap restoranı bulunamadı.")
    print("💡 Daha az kriter ile tekrar deneyin.")

# Test 3: İzmir Konak'ta pizza menüsü olan restoranlar
print("\n🔥 TEST 3: İzmir Konak'ta pizza menüsü olan restoranlar")
criteria3 = {
    "city": "izmir",
    "district": "konak",
    "menu": "pizza",
    "limit": 3
}
results3 = chatbot_query_restaurants(kg, criteria3, limit=5) if kg else []
if results3:
    print(format_chatbot_response_simple(results3))  # Sonuçları formatla ve sadece isimleri göster
else:
    print("❌ İzmir Konak'ta pizza menüsü olan restoran bulunamadı.")
    print("💡 Daha az kriter ile tekrar deneyin.")


🎯 CHATBOT TABANLI ÇOKLU RELATION TESTLERİ
✅ Neo4j bağlantısı aktif

🔥 TEST 1: Ankara'da kebap menüsü ve çocuk parkı olan restoranlar
❌ Bu kriterlere uygun restoran bulunamadı.
💡 Daha az kriter ile tekrar deneyin.

🔥 TEST 2: İstanbul'da romantik ve yüksek puanlı kebap restoranları
✨ Önerilen mekanlar:
1. papuli_restaurant
2. arca_restaurant_grill
3. roof_mezze_360_restaurant
4. fb_culture
5. beso_rooftop_restaurant

🔥 TEST 3: İzmir Konak'ta pizza menüsü olan restoranlar
✨ Önerilen mekanlar:
1. la_fourmi_restaurant
2. favino_alsancak


-----------------------------------------

In [ ]:
import re

def parse_user_input(user_input):
    """
    Kullanıcının serbest metnini, restoran arama kriterlerine ayıran fonksiyon.
    """
    user_input = user_input.lower()  # Cümleyi küçük harfe çeviriyoruz

    # Başlangıçta boş değerler
    city = None
    district = None
    menu = None
    occasion = None
    amenities = []
    min_score = None

    # Şehir için anahtar kelimeler
    cities = ["ankara", "istanbul", "izmir"]
    for city_name in cities:
        if city_name in user_input:
            city = city_name
            break

    # İlçe için anahtar kelimeler
    districts = ["kadıköy", "beşiktaş", "şişli", "fatih", "üsküdar", "bakırköy", "beyoğlu", "maltepe", "kartal", "ataşehir", "bağcılar", "zeytinburnu", "avcılar",
                "çankaya", "keçiören", "mamak", "etimesgut", "sincan", "yenimahalle", "altındağ", "çubuk", "polatlı", "gölbaşı", "pursaklar", "bala",
                "konak", "karşıyaka", "bornova", "buca", "çiğli", "alsancak", "gaziemir", "bayraklı", "narlıdere", "menemen", "foça", "urla"]
    for district_name in districts:
        if district_name in user_input:
            district = district_name
            break

    # Menü öğeleri
    menu_items = ["kebap", "pizza", "burger", "mantı", "adana dürüm", "kahvaltı", "tatlı", "çorba", "meze", "salata", "baklava", "künefe", "sushi", "ramen", "pizza", "hamburger", "burger", "biftek", "falafel", "humus", "shawarma", "taco", "burrito", "curry", "penne", "spaghetti", "risotto", "gnocchi", "türk mutfağı", "geleneksel türk mutfağı", "kahvaltı", "tatlı"]
    for item in menu_items:
        if item in user_input:
            menu = item
            break

    # Özel günler
    occasions = ["romantik", "aile", "iş yemeği", "doğum günü", "tatil", "sevgililer günü"]
    for occasion_type in occasions:
        if occasion_type in user_input:
            occasion = occasion_type
            break

    # İmkanlar
    amenity_keywords = ["bahçe", "çocuk parkı", "otopark", "wi-fi", "canlı müzik", "deniz manzarası", "teras", "açık alan", "açık hava"]
    for amenity in amenity_keywords:
        if amenity in user_input:
            amenities.append(amenity)

    # Min score (puan) için regex
    score_match = re.search(r"\b(\d\.\d)\b", user_input)
    if score_match:
        min_score = float(score_match.group(1))

    # Kriterleri dictionary'ye yerleştiriyoruz
    criteria = {
        "city": city,
        "district": district,
        "menu": menu,
        "occasion": occasion,
        "amenities": amenities,
        "min_score": min_score,
        "limit": 5
    }

    return criteria

def chatbot_interface(kg, data):
    print("🎯 Merhaba! Restoran arama chatbot'uma hoş geldiniz.")
    print("=" * 60)

    while True:
        print("\nLütfen restoran arama kriterlerinizi tek bir cümle olarak girin:")

        # Kullanıcıdan serbest metin alın
        user_input = input("Restoran arama cümlenizi girin (örneğin: 'İstanbul'da romantik kebapçılar' veya 'Ankara'da kebap yiyebileceğim yerler'): ")

        # Cümleyi parse et
        criteria = parse_user_input(user_input)

        # Restoranları arıyoruz
        results = chatbot_query_restaurants(kg, criteria, limit=5)

        if results:
            # Sonuçları formatla
            formatted_response = format_chatbot_response(results, data)
            print("\n🎯 Restoranlar:")
            print(formatted_response)
        else:
            print("❌ Bu kriterlere uygun restoran bulunamadı.")

        # Yeni bir arama yapmak isteyip istemediğini sor
        again = input("\nYeni bir arama yapmak ister misiniz? (Evet/Hayır): ").strip().lower()
        if again != "evet":
            print("👋 Teşekkürler! İyi günler dilerim.")
            break

def chatbot_query_restaurants(kg, criteria, limit=5):
    if kg is None:
        return [{"error": "Neo4j bağlantısı yok"}]

    with kg.driver.session() as session:
        match_clauses = ["(r:Restaurant)"]
        conditions = []
        params = {"limit": limit}

        # Şehir
        if "city" in criteria and criteria["city"]:
            match_clauses.append("(r)-[:LOCATED_IN]->(city:Location)")
            conditions.append("toLower(city.name) = $city")
            params["city"] = criteria["city"]

        # İlçe
        if "district" in criteria and criteria["district"]:
            match_clauses.append("(r)-[:LOCATED_IN]->(district:Location)")
            conditions.append("toLower(district.name) = $district")
            params["district"] = criteria["district"]

        # Menü
        if "menu" in criteria and criteria["menu"]:
            match_clauses.append("(r)-[:SERVES]->(m:MenuItem)")
            conditions.append("toLower(m.name) CONTAINS $menu")
            params["menu"] = criteria["menu"]

        # Özel gün
        if "occasion" in criteria and criteria["occasion"]:
            match_clauses.append("(r)-[:SUITS_OCCASION]->(o:Occasion)")
            conditions.append("toLower(o.name) = $occasion")
            params["occasion"] = criteria["occasion"]

        # İmkanlar
        if "amenities" in criteria and criteria["amenities"]:
            for i, amenity in enumerate(criteria["amenities"]):
                match_clauses.append(f"(r)-[:HAS_AMENITY]->(a{i}:Amenity)")
                conditions.append(f"toLower(a{i}.name) CONTAINS $amenity{i}")
                params[f"amenity{i}"] = amenity

        # Min puan
        if "min_score" in criteria and criteria["min_score"]:
            match_clauses.append("(r)-[rating:RATED_FOR]->(asp:Aspect)")
            conditions.append("rating.score >= $min_score")
            params["min_score"] = criteria["min_score"]

        match_clause = "MATCH " + ", ".join(match_clauses)
        where_clause = "WHERE " + " AND ".join(conditions) if conditions else ""

        query = f"""
        {match_clause}
        {where_clause}
        OPTIONAL MATCH (r)-[rt:RATED_FOR]->(asp:Aspect)
        OPTIONAL MATCH (r)-[:HAS_AMENITY]->(am:Amenity)
        OPTIONAL MATCH (r)-[:SERVES]->(m:MenuItem)
        OPTIONAL MATCH (r)-[:SUITS_OCCASION]->(o:Occasion)
        WHERE rt.sentiment = 'positive'
        WITH r,
             avg(rt.score) as avg_score,
             collect(DISTINCT asp.name) as aspects,
             collect(DISTINCT am.name) as amenities,
             collect(DISTINCT m.name) as menu_items,
             collect(DISTINCT o.name) as occasions
        RETURN r.name as restoran,
               round(avg_score,2) as ortalama_puan,
               aspects, amenities, menu_items, occasions
        ORDER BY ortalama_puan DESC
        LIMIT $limit
        """

        result = session.run(query, params)
        return [dict(record) for record in result]

kg = Neo4jKnowledgeGraph(uri="bolt://localhost:7687", user="neo4j", password="12345678")
chatbot_interface(kg, data)


🎯 Merhaba! Restoran arama chatbot'uma hoş geldiniz.

Lütfen restoran arama kriterlerinizi tek bir cümle olarak girin:

🎯 Restoranlar:
✨ Önerilen mekanlar:
1. vento_italiano_ristorante (Puan: None) | Özet: Hata: Error code: 400 - {'error': 'Trying to keep the first 8813 tokens when context the overflows. However, the model is loaded with context length of only 8192 tokens, which is not enough. Try to load the model with a larger context length, or provide a shorter input'}
2. delicatessen_etiler (Puan: None) | Özet: Hata: Error code: 400 - {'error': 'Trying to keep the first 8338 tokens when context the overflows. However, the model is loaded with context length of only 8192 tokens, which is not enough. Try to load the model with a larger context length, or provide a shorter input'}
3. carsi_ici_merkez_lokantasi (Puan: None) | Özet: Restoran, müşteriler tarafından "harika" olarak nitelendiriliyor ve bu olumlu geri bildirimle öne çıkıyor.
4. aykoc_restoran (Puan: None) | Özet: Merhaba! 😊

-------------------------------

# For Rubric Test

In [ ]:
import pandas as pd

def run_queries_and_store_results(kg):
    # Sorguları tanımlıyoruz
    queries = [
        {
            "name": "Ankara'daki mekanlar",
            "query": """
            MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
            WHERE toLower(l.name) = 'ankara'
            RETURN r.name as restoran, l.name as lokasyon
            LIMIT 10
            """
        },

        {
            "name": "Olumlu tat puanı alan restoranlar",
            "query": """
            MATCH (r:Restaurant)-[rating:RATED_FOR]->(a:Aspect)
            WHERE rating.dimension = 'TASTE' AND rating.sentiment = 'positive'
            RETURN r.name as restoran, a.name as aspekt, rating.score as puan
            ORDER BY rating.score DESC
            LIMIT 10
            """
        },

        {
            "name": "Restoranların imkanları",
            "query": """
            MATCH (r:Restaurant)-[:HAS_AMENITY]->(am:Amenity)
            RETURN r.name as restoran, collect(am.name) as imkanlar
            LIMIT 10
            """
        },

        {
            "name": "İstanbul'da romantik ve yüksek tat puanı olan restoranlar",
            "query": """
            MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
            MATCH (r)-[rating:RATED_FOR]->(a:Aspect)
            MATCH (r)-[:SUITS_OCCASION]->(o:Occasion)
            WHERE toLower(l.name) = 'istanbul'
              AND rating.dimension = 'TASTE'
              AND rating.sentiment = 'positive'
              AND toLower(o.name) = 'romantik'
            RETURN r.name as restoran, avg(rating.score) as ortalama_puan
            ORDER BY ortalama_puan DESC
            LIMIT 10
            """
        },
                {
            "name": "Ankara'da bahçeli kebapçılar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[:HAS_AMENITY]->(am:Amenity)
                WHERE toLower(l.name) CONTAINS 'ankara'
                AND toLower(am.name) CONTAINS 'bahçe'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "Ankara'da temiz kebapçılar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[rating:RATED_FOR]->(a:Aspect)
                WHERE toLower(l.name) CONTAINS 'ankara'
                AND rating.dimension = 'CLEANLINESS'
                AND rating.sentiment = 'positive'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "Ankara'da hızlı servisli dönerci",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[rating:RATED_FOR]->(a:Aspect)
                WHERE toLower(l.name) CONTAINS 'ankara'
                AND rating.dimension = 'SERVICE'
                AND rating.sentiment = 'positive'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "Ankara'da otoparklı restoranlar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[:HAS_AMENITY]->(am:Amenity)
                WHERE toLower(l.name) CONTAINS 'ankara'
                AND toLower(am.name) CONTAINS 'otopark'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "Ankara'da çocuk oyun alanlı mekanlar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[:HAS_AMENITY]->(am:Amenity)
                WHERE toLower(l.name) CONTAINS 'ankara'
                AND toLower(am.name) CONTAINS 'çocuk'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "Ankara'da açık mutfaklı kafeler",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[:HAS_AMENITY]->(am:Amenity)
                WHERE toLower(l.name) CONTAINS 'ankara'
                AND toLower(am.name) CONTAINS 'açık mutfak'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "İzmir'de deniz manzaralı balıkçılar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[:HAS_AMENITY]->(am:Amenity)
                WHERE toLower(l.name) CONTAINS 'izmir'
                AND toLower(am.name) CONTAINS 'manzara'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "İzmir'de sessiz balıkçılar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[rating:RATED_FOR]->(a:Aspect)
                WHERE toLower(l.name) CONTAINS 'izmir'
                AND rating.dimension = 'AMBIANCE'
                AND rating.sentiment = 'positive'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "İzmir'de canlı müzikli meyhane",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[:HAS_AMENITY]->(am:Amenity)
                WHERE toLower(l.name) CONTAINS 'izmir'
                AND toLower(am.name) CONTAINS 'müzik'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "İzmir'de bahçeli kebapçılar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[:HAS_AMENITY]->(am:Amenity)
                WHERE toLower(l.name) CONTAINS 'izmir'
                AND toLower(am.name) CONTAINS 'bahçe'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "İzmir'de açık alanlı pizzacılar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[:HAS_AMENITY]->(am:Amenity)
                WHERE toLower(l.name) CONTAINS 'izmir'
                AND toLower(am.name) CONTAINS 'açık alan'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "İzmir'de aile salonu olan lokantalar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[:HAS_AMENITY]->(am:Amenity)
                WHERE toLower(l.name) CONTAINS 'izmir'
                AND toLower(am.name) CONTAINS 'aile'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "İstanbul'da deniz manzaralı kahvaltıcılar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[:HAS_AMENITY]->(am:Amenity)
                WHERE toLower(l.name) CONTAINS 'istanbul'
                AND toLower(am.name) CONTAINS 'manzara'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "İstanbul'da sessiz restoranlar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[rating:RATED_FOR]->(a:Aspect)
                WHERE toLower(l.name) CONTAINS 'istanbul'
                AND rating.dimension = 'AMBIANCE'
                AND rating.sentiment = 'positive'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "İstanbul'da otoparklı kebapçılar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[:HAS_AMENITY]->(am:Amenity)
                WHERE toLower(l.name) CONTAINS 'istanbul'
                AND toLower(am.name) CONTAINS 'otopark'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "İstanbul'da hızlı servisli pizzacılar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[rating:RATED_FOR]->(a:Aspect)
                WHERE toLower(l.name) CONTAINS 'istanbul'
                AND rating.dimension = 'SERVICE'
                AND rating.sentiment = 'positive'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "İstanbul'da balık menüsü geniş restoranlar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[rating:RATED_FOR]->(a:Aspect)
                WHERE toLower(l.name) CONTAINS 'istanbul'
                AND rating.dimension = 'FOOD_VARIETY'
                AND rating.sentiment = 'positive'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "İstanbul'da sigarasız kapalı alan restoranlar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[:HAS_AMENITY]->(am:Amenity)
                WHERE toLower(l.name) CONTAINS 'istanbul'
                AND toLower(am.name) CONTAINS 'sigarasız'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "Ankara'da Adana kebap yapan restoranlar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[rating:RATED_FOR]->(a:Aspect)
                WHERE toLower(l.name) CONTAINS 'ankara'
                AND rating.dimension = 'FOOD_VARIETY'
                AND rating.sentiment = 'positive'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "Ankara'da mantı yapan mekanlar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[rating:RATED_FOR]->(a:Aspect)
                WHERE toLower(l.name) CONTAINS 'ankara'
                AND rating.dimension = 'FOOD_VARIETY'
                AND rating.sentiment = 'positive'
                RETURN r.name as restoran
                LIMIT 10
            """
        },

        # 21-40: Menu/Cuisine focused queries
        {
            "name": "İzmir'de ince hamur pizzacılar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[rating:RATED_FOR]->(a:Aspect)
                WHERE toLower(l.name) CONTAINS 'izmir'
                AND rating.dimension = 'TASTE'
                AND rating.sentiment = 'positive'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "İzmir'de tatlı menüsü geniş kafeler",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[rating:RATED_FOR]->(a:Aspect)
                WHERE toLower(l.name) CONTAINS 'izmir'
                AND rating.dimension = 'FOOD_VARIETY'
                AND rating.sentiment = 'positive'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "İzmir'de vejetaryen menüsü geniş restoranlar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[rating:RATED_FOR]->(a:Aspect)
                WHERE toLower(l.name) CONTAINS 'izmir'
                AND rating.dimension = 'FOOD_VARIETY'
                AND rating.sentiment = 'positive'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "İzmir'de sushi menüsü olan mekanlar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[rating:RATED_FOR]->(a:Aspect)
                WHERE toLower(l.name) CONTAINS 'izmir'
                AND rating.dimension = 'FOOD_VARIETY'
                AND rating.sentiment = 'positive'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "İstanbul'da vegan kahvaltıcılar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[rating:RATED_FOR]->(a:Aspect)
                WHERE toLower(l.name) CONTAINS 'istanbul'
                AND rating.dimension = 'FOOD_VARIETY'
                AND rating.sentiment = 'positive'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "İstanbul'da İtalyan mutfağı restoranlar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[rating:RATED_FOR]->(a:Aspect)
                WHERE toLower(l.name) CONTAINS 'istanbul'
                AND rating.dimension = 'FOOD_VARIETY'
                AND rating.sentiment = 'positive'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "İstanbul'da hamburger yapan dükkânlar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[rating:RATED_FOR]->(a:Aspect)
                WHERE toLower(l.name) CONTAINS 'istanbul'
                AND rating.dimension = 'FOOD_VARIETY'
                AND rating.sentiment = 'positive'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "İstanbul'da Lübnan mutfağı restoranlar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[rating:RATED_FOR]->(a:Aspect)
                WHERE toLower(l.name) CONTAINS 'istanbul'
                AND rating.dimension = 'FOOD_VARIETY'
                AND rating.sentiment = 'positive'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "Ankara'da etli ekmek yapan lokantalar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[rating:RATED_FOR]->(a:Aspect)
                WHERE toLower(l.name) CONTAINS 'ankara'
                AND rating.dimension = 'FOOD_VARIETY'
                AND rating.sentiment = 'positive'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "Tüm şehirlerde şarap menüsü geniş restoranlar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[rating:RATED_FOR]->(a:Aspect)
                WHERE rating.dimension = 'BEVERAGE_VARIETY'
                AND rating.sentiment = 'positive'
                RETURN r.name as restoran, l.name as sehir
                LIMIT 10
            """
        },
        {
            "name": "En yüksek tat puanı alan restoranlar",
            "query": """
                MATCH (r:Restaurant)-[rating:RATED_FOR]->(a:Aspect)
                WHERE rating.dimension = 'TASTE' AND rating.sentiment = 'positive'
                RETURN r.name as restoran, rating.score as puan
                ORDER BY rating.score DESC
                LIMIT 10
            """
        },
        {
            "name": "En yüksek servis puanı alan restoranlar",
            "query": """
                MATCH (r:Restaurant)-[rating:RATED_FOR]->(a:Aspect)
                WHERE rating.dimension = 'SERVICE' AND rating.sentiment = 'positive'
                RETURN r.name as restoran, rating.score as puan
                ORDER BY rating.score DESC
                LIMIT 10
            """
        },
        {
            "name": "En temiz restoranlar",
            "query": """
                MATCH (r:Restaurant)-[rating:RATED_FOR]->(a:Aspect)
                WHERE rating.dimension = 'CLEANLINESS' AND rating.sentiment = 'positive'
                RETURN r.name as restoran, rating.score as puan
                ORDER BY rating.score DESC
                LIMIT 10
            """
        },
        {
            "name": "En iyi ambiyans puanı alan mekanlar",
            "query": """
                MATCH (r:Restaurant)-[rating:RATED_FOR]->(a:Aspect)
                WHERE rating.dimension = 'AMBIANCE' AND rating.sentiment = 'positive'
                RETURN r.name as restoran, rating.score as puan
                ORDER BY rating.score DESC
                LIMIT 10
            """
        },
        {
            "name": "En çok menü çeşitliliği olan restoranlar",
            "query": """
                MATCH (r:Restaurant)-[rating:RATED_FOR]->(a:Aspect)
                WHERE rating.dimension = 'FOOD_VARIETY' AND rating.sentiment = 'positive'
                RETURN r.name as restoran, rating.score as puan
                ORDER BY rating.score DESC
                LIMIT 10
            """
        },
        {
            "name": "Düşük tat puanı alan restoranlar",
            "query": """
                MATCH (r:Restaurant)-[rating:RATED_FOR]->(a:Aspect)
                WHERE rating.dimension = 'TASTE' AND rating.sentiment = 'negative'
                RETURN r.name as restoran, rating.score as puan
                ORDER BY rating.score ASC
                LIMIT 10
            """
        },
        {
            "name": "Kötü servis puanı alan restoranlar",
            "query": """
                MATCH (r:Restaurant)-[rating:RATED_FOR]->(a:Aspect)
                WHERE rating.dimension = 'SERVICE' AND rating.sentiment = 'negative'
                RETURN r.name as restoran, rating.score as puan
                ORDER BY rating.score ASC
                LIMIT 10
            """
        },
        {
            "name": "Hijyen problemi olan restoranlar",
            "query": """
                MATCH (r:Restaurant)-[rating:RATED_FOR]->(a:Aspect)
                WHERE rating.dimension = 'CLEANLINESS' AND rating.sentiment = 'negative'
                RETURN r.name as restoran, rating.score as puan
                ORDER BY rating.score ASC
                LIMIT 10
            """
        },
        {
            "name": "Gürültülü ambiyans şikayeti olan mekanlar",
            "query": """
                MATCH (r:Restaurant)-[rating:RATED_FOR]->(a:Aspect)
                WHERE rating.dimension = 'AMBIANCE' AND rating.sentiment = 'negative'
                RETURN r.name as restoran, rating.score as puan
                ORDER BY rating.score ASC
                LIMIT 10
            """
        },
        {
            "name": "Fiyat şikayeti olan restoranlar",
            "query": """
                MATCH (r:Restaurant)-[rating:RATED_FOR]->(a:Aspect)
                WHERE rating.dimension = 'VALUE' AND rating.sentiment = 'negative'
                RETURN r.name as restoran, rating.score as puan
                ORDER BY rating.score ASC
                LIMIT 10
            """
        },

        # 41-60: Occasion-based queries
        {
            "name": "Romantik akşam yemeği için uygun mekanlar",
            "query": """
                MATCH (r:Restaurant)-[:SUITS_OCCASION]->(o:Occasion)
                WHERE toLower(o.name) CONTAINS 'romantik'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "İş yemeği için uygun restoranlar",
            "query": """
                MATCH (r:Restaurant)-[:SUITS_OCCASION]->(o:Occasion)
                WHERE toLower(o.name) CONTAINS 'iş'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "Doğum günü kutlaması için uygun mekanlar",
            "query": """
                MATCH (r:Restaurant)-[:SUITS_OCCASION]->(o:Occasion)
                WHERE toLower(o.name) CONTAINS 'doğum günü'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "Nişan yemeği için uygun restoranlar",
            "query": """
                MATCH (r:Restaurant)-[:SUITS_OCCASION]->(o:Occasion)
                WHERE toLower(o.name) CONTAINS 'nişan'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "Mezuniyet yemeği için uygun mekanlar",
            "query": """
                MATCH (r:Restaurant)-[:SUITS_OCCASION]->(o:Occasion)
                WHERE toLower(o.name) CONTAINS 'mezuniyet'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "Yılbaşı kutlaması için uygun restoranlar",
            "query": """
                MATCH (r:Restaurant)-[:SUITS_OCCASION]->(o:Occasion)
                WHERE toLower(o.name) CONTAINS 'yılbaşı'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "Arkadaş buluşması için uygun mekanlar",
            "query": """
                MATCH (r:Restaurant)-[:SUITS_OCCASION]->(o:Occasion)
                WHERE toLower(o.name) CONTAINS 'arkadaş'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "Aile yemeği için uygun restoranlar",
            "query": """
                MATCH (r:Restaurant)-[:SUITS_OCCASION]->(o:Occasion)
                WHERE toLower(o.name) CONTAINS 'aile'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "Ankara'da romantik mekanlar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[:SUITS_OCCASION]->(o:Occasion)
                WHERE toLower(l.name) CONTAINS 'ankara'
                AND toLower(o.name) CONTAINS 'romantik'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "İzmir'de romantik mekanlar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[:SUITS_OCCASION]->(o:Occasion)
                WHERE toLower(l.name) CONTAINS 'izmir'
                AND toLower(o.name) CONTAINS 'romantik'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "İstanbul'da romantik mekanlar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[:SUITS_OCCASION]->(o:Occasion)
                WHERE toLower(l.name) CONTAINS 'istanbul'
                AND toLower(o.name) CONTAINS 'romantik'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "Ankara'da iş yemeği için uygun mekanlar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[:SUITS_OCCASION]->(o:Occasion)
                WHERE toLower(l.name) CONTAINS 'ankara'
                AND toLower(o.name) CONTAINS 'iş'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "İzmir'de iş yemeği için uygun mekanlar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[:SUITS_OCCASION]->(o:Occasion)
                WHERE toLower(l.name) CONTAINS 'izmir'
                AND toLower(o.name) CONTAINS 'iş'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "İstanbul'da iş yemeği için uygun mekanlar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[:SUITS_OCCASION]->(o:Occasion)
                WHERE toLower(l.name) CONTAINS 'istanbul'
                AND toLower(o.name) CONTAINS 'iş'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "Ankara'da doğum günü kutlaması için mekanlar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[:SUITS_OCCASION]->(o:Occasion)
                WHERE toLower(l.name) CONTAINS 'ankara'
                AND toLower(o.name) CONTAINS 'doğum günü'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "İzmir'de doğum günü kutlaması için mekanlar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[:SUITS_OCCASION]->(o:Occasion)
                WHERE toLower(l.name) CONTAINS 'izmir'
                AND toLower(o.name) CONTAINS 'doğum günü'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "İstanbul'da doğum günü kutlaması için mekanlar",
            "query": """
                MATCH (r:Restaurant)-[:LOCATED_IN]->(l:Location)
                MATCH (r)-[:SUITS_OCCASION]->(o:Occasion)
                WHERE toLower(l.name) CONTAINS 'istanbul'
                AND toLower(o.name) CONTAINS 'doğum günü'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "Loş ışıklı romantik restoranlar",
            "query": """
                MATCH (r:Restaurant)-[:HAS_AMENITY]->(am:Amenity)
                MATCH (r)-[:SUITS_OCCASION]->(o:Occasion)
                WHERE toLower(am.name) CONTAINS 'loş ışık'
                AND toLower(o.name) CONTAINS 'romantik'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "Süsleme yapan doğum günü mekanları",
            "query": """
                MATCH (r:Restaurant)-[:HAS_AMENITY]->(am:Amenity)
                MATCH (r)-[:SUITS_OCCASION]->(o:Occasion)
                WHERE toLower(am.name) CONTAINS 'süsleme'
                AND toLower(o.name) CONTAINS 'doğum günü'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "Pasta servisi olan kafeler",
            "query": """
                MATCH (r:Restaurant)-[:HAS_AMENITY]->(am:Amenity)
                WHERE toLower(am.name) CONTAINS 'pasta'
                RETURN r.name as restoran
                LIMIT 10
            """
        },

        # 61-80: Amenity-focused queries
        {
            "name": "Restoranların imkanları",
            "query": """
                MATCH (r:Restaurant)-[:HAS_AMENITY]->(am:Amenity)
                RETURN r.name as restoran, collect(am.name) as imkanlar
                LIMIT 10
            """
        },
        {
            "name": "En çok amenity'ye sahip restoranlar",
            "query": """
                MATCH (r:Restaurant)-[:HAS_AMENITY]->(am:Amenity)
                RETURN r.name as restoran, count(am) as amenity_sayisi
                ORDER BY amenity_sayisi DESC
                LIMIT 10
            """
        },
        {
            "name": "WiFi olan kafeler",
            "query": """
                MATCH (r:Restaurant)-[:HAS_AMENITY]->(am:Amenity)
                WHERE toLower(am.name) CONTAINS 'wifi'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "Klimalı restoranlar",
            "query": """
                MATCH (r:Restaurant)-[:HAS_AMENITY]->(am:Amenity)
                WHERE toLower(am.name) CONTAINS 'klima'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "Vale park hizmeti olan mekanlar",
            "query": """
                MATCH (r:Restaurant)-[:HAS_AMENITY]->(am:Amenity)
                WHERE toLower(am.name) CONTAINS 'vale'
                RETURN r.name as restoran
                LIMIT 10
            """
        },
        {
            "name": "Kart ile ödeme kabul eden restoranlar",
            "query": """
                MATCH (r:Restaurant)-[:HAS_AMENITY]->(am:Amenity)
                WHERE toLower(am.name) CONTAINS 'kart'
                RETURN r.name as restoran
                LIMIT 10
            """
        },

    ]

    # DataFrame oluşturuluyor
    all_results = []  # Sonuçları depolayacağımız liste

    with kg.driver.session() as session:
        for q in queries:
            input_query = q["name"]
            output_results = []  # Çıktıları depolayacağımız liste

            print(f"Sorgu: {input_query}")

            try:
                result = session.run(q['query'])
                records = list(result)

                if records:
                    print(f"   ✓ {len(records)} sonuç bulundu:")
                    for record in records:
                        output_results.append(record["restoran"])  # Restoran adlarını ekle
                else:
                    print("   ⚠ Sonuç bulunamadı")

            except Exception as e:
                print(f"   ❌ Sorgu hatası: {e}")

            # Input ve Output'u DataFrame'e ekliyoruz
            for result in output_results:
                all_results.append([input_query, result])

    # DataFrame oluşturuluyor
    df = pd.DataFrame(all_results, columns=["Input", "Output"])

    return df



In [ ]:
# Neo4j bağlantısını başlatıyoruz
kg = Neo4jKnowledgeGraph(uri="bolt://localhost:7687", user="neo4j", password="12345678")
df_results = run_queries_and_store_results(kg)

# Sonuçları yazdır
print(df_results)


Sorgu: Ankara'daki mekanlar
   ✓ 10 sonuç bulundu:
Sorgu: Olumlu tat puanı alan restoranlar
   ✓ 10 sonuç bulundu:
Sorgu: Restoranların imkanları
   ✓ 10 sonuç bulundu:
Sorgu: İstanbul'da romantik ve yüksek tat puanı olan restoranlar
   ✓ 10 sonuç bulundu:
                                                Input  \
0                                Ankara'daki mekanlar   
1                                Ankara'daki mekanlar   
2                                Ankara'daki mekanlar   
3                                Ankara'daki mekanlar   
4                                Ankara'daki mekanlar   
5                                Ankara'daki mekanlar   
6                                Ankara'daki mekanlar   
7                                Ankara'daki mekanlar   
8                                Ankara'daki mekanlar   
9                                Ankara'daki mekanlar   
10                  Olumlu tat puanı alan restoranlar   
11                  Olumlu tat puanı alan restoranlar   
12

In [ ]:
df_results

,Input,Output
0,Ankara'daki mekanlar,beyzade_cafe_elmadag
1,Ankara'daki mekanlar,bolu_mangal_evi
2,Ankara'daki mekanlar,cafemodern
3,Ankara'daki mekanlar,annem_pide_ve_kebap_salonu
4,Ankara'daki mekanlar,akin_usta_iskender
5,Ankara'daki mekanlar,can_dadas_erzurum_cag_kebabi
6,Ankara'daki mekanlar,akdeniz_iskembecisi
7,Ankara'daki mekanlar,bey_aspava
8,Ankara'daki mekanlar,babanin_yeri_lokantasi
9,Ankara'daki mekanlar,altinova_pide_kebap_salonu
